In [1]:
import numpy as np
from scipy.integrate import quad
from scipy.special import erfc,erf

In [2]:
def dbexp(x):
    if x < -100:
        return 0.0
    elif x > 700:  # avoid overflow
        return np.exp(700.0)
    else:
        return np.exp(x)

def log_transform_integrand(func):
    def transformed_func(x):
        return dbexp(x) * func(dbexp(x))
    return transformed_func

def romb_quad(func, xmin, xmax, stoper=1.0e-9):
    """Perform Romberg integration on log-transformed interval using scipy quad"""
    if xmin >= xmax:
        return 0.0

    xln1 = np.log(xmin)
    xln2 = np.log(xmax)
    transformed_func = log_transform_integrand(func)
    
    result, error = quad(transformed_func, xln1, xln2, epsabs=stoper)
    return result

In [3]:
def romb(func, xmin, xmax, level=15, stoper = 1.0e-9):
    """Perform Romberg integration on log-transformed interval"""
    if xmin >= xmax:
        return 0.0

    xln1 = np.log(xmin)
    xln2 = np.log(xmax)
    dx1 = xln2 - xln1
    sum_val = 0.5 * xmin * func(xmin)
    area = sum_val * dx1
    r = np.zeros((level+1, level+1))
    r[1, 1] = area
    dx2 = dx1 / 2.0
    inc = 1

    for i in range(2, level+1):
        xln = xln1 + dx2
        for m in range(inc):
            xe = dbexp(xln)
            sum_val += xe * func(xe)
            xln += dx1
        area = sum_val * dx2
        r[1, i] = area
        for j in range(2, i + 1):
            k = i + 1 - j
            r[j, k] = (4.0 ** (j - 1) * r[j - 1, k + 1] - r[j - 1, k]) / (4.0 ** (j - 1) - 1)
        if r[i, 1] > 0.0:
            error = abs((r[i, 1] - r[i - 1, 2]) / r[i, 1])
            if (error < stoper and i > 4)  :
                return area
            if (error < stoper * 100.0 and abs(area) < 1.0e-10):
                return area
        dx1 /= 2.0
        dx2 /= 2.0
        inc *= 2

    return area

In [4]:
def chebylog(func, aa, bb, m = 100):
    """Perform integration of f(x) between a and b using M-point Gauss-Chebyshev quadrature formula"""
    a = np.log(aa)
    b = np.log(bb)

    sum_val = 0.0
    for i in range(1, m + 1):
        z1 = np.cos((2 * (i - 1) + 1) * np.pi / (2 * m))
        x1 = (z1 * (b - a) + b + a) / 2.0
        dx1 = dbexp(x1)
        sum_val += dx1 * func(dx1) * np.sqrt(1.0 - z1 * z1)

    return (b - a) * np.pi * sum_val / (2 * m)


In [5]:
xmin = np.pi/8
xmax = 2*np.pi
result = romb_quad(erfc, xmin, xmax)

In [6]:
result

0.25632652860603233

In [7]:
result = romb(erfc, xmin, xmax)

In [8]:
result

0.2563143126933558

In [9]:
result = chebylog(erfc, xmin, xmax)

In [10]:
result

0.2563394828798779

In [12]:
result, error = quad(erfc, xmin, xmax)

In [13]:
result

0.25632652860603233